## 第一题

In [1]:
import xlrd
file_path = r'C:/Users/HP/Desktop/kbxj.xlsx'
#获取数据
data = xlrd.open_workbook(file_path)
#获取sheet
table = data.sheet_by_name('Data')

In [6]:
#第一题数据处理
def creat_data_1(table):
    data = []
    nrows = table.nrows
    for i in range(2,nrows):
        #获取一行的数值
        row = []
        rowvalue = table.row_values(i)
        if rowvalue[5] == '':              #是否持续
            rowvalue[5] = 0
        row.append(rowvalue[5])
        if rowvalue[19] == '':              #是否入选政治宗教
            rowvalue[19] = 0
        row.append(rowvalue[19])
        if rowvalue[20] == '':              #恐吓煽动更多群众
            rowvalue[20] = 0
        row.append(rowvalue[20])
        if rowvalue[21] == '':              #超出人道主义
            rowvalue[21] = 0
        row.append(rowvalue[21])
        if rowvalue[22] == '':              #恐怖主义
            rowvalue[22] = 0
        row.append(rowvalue[22])
        
        if rowvalue[28] in [2,3,4]:              #攻击类型   <武装袭击 爆炸  劫持>
            rowvalue[28] = 1
        else:
            rowvalue[28] = 0
        row.append(rowvalue[28])
    
        if rowvalue[81] in [7,8,9,10,11,12,13,14,15,16,17,28,29]:              #武器类型
            rowvalue[81] = 1
        else:
            rowvalue[81] = 0
        row.append(rowvalue[81])

        if rowvalue[98] =='' :             
            rowvalue[98] = 0
        elif rowvalue[98] <10 :              #死亡人数      
            rowvalue[98] = 1
        elif rowvalue[98] <50 :              
            rowvalue[98] = 2
        elif rowvalue[98] <100 :             
            rowvalue[98] = 3
        elif rowvalue[98] <300 :              
            rowvalue[98] = 4
        else :             
            rowvalue[98] = 5
        row.append(rowvalue[98])   
        
        if  rowvalue[101] =='':
            rowvalue[101] =0
        elif rowvalue[101] <10 :              #受伤人数        
            rowvalue[101] = 1
        elif rowvalue[101] <50 :              
            rowvalue[101] = 2
        elif rowvalue[101] <100 :              
            rowvalue[101] = 3
        elif rowvalue[101] <300 :             
            rowvalue[101] = 4
        else :              
            rowvalue[101] = 5
        row.append(rowvalue[101])    
        
        if rowvalue[105] ==1:                      
            rowvalue[105] = 5
        elif rowvalue[105] ==2 :              
            rowvalue[105] = 2
        else:              
            rowvalue[105] = 1
        row.append(rowvalue[105])
        data.append(row)
    return data

In [7]:
# 第一题 计算  最大差 和  最小值
def mx_mn(data):
    mm = []
    for i in range(len(data[0])):
        m = [0,0]
        max = 0
        min = 0
        for j in range(len(data)):
            if data[j][i]>max:
                max = data[j][i]
            if data[j][i]<min:
                min = data[j][i]

        max_min = max-min
        m[0] = max_min
        m[1] = min
        mm.append(m)
    return mm

In [8]:
#  第一题数据归一化
def redata(data,ls):
    for u,i in enumerate(data):
        for n,j in enumerate(i):
            e = (j-ls[n][1])/ls[n][0]
            data[u][n] = e
    return data

In [9]:
# 第一题 聚类
from collections import defaultdict
from random import uniform
from math import sqrt

#  加权 欧式距离计算
def distance(a,b):
    dimention=len(a)
    sum=0
    for i in range(dimention):
        if i<7:
            sq=((a[i]-b[i])**2)/16
        if i ==7:
            sq=5*(a[i]-b[i])**2
        if i ==8:
            sq=4*(a[i]-b[i])**2
        if i == 9:
            sq=3*(a[i]-b[i])**2
        sum+=sq
    return sqrt(sum)

# 聚类
def assign_points(data_points,centers):
    assignments=[]
    for point in data_points:
        shortest=float('inf')
        shortest_index = 0
        for i in range(len(centers)):
            value=distance(point,centers[i])
            if value<shortest:
                shortest=value
                shortest_index=i
        assignments.append(shortest_index)
    return assignments

def point_avg(points):
    dimensions=len(points[0])
    new_center=[]
    for dimension in range(dimensions):
        sum=0
        for p in points:
            sum+=p[dimension]
        new_center.append(float("%.8f"%(sum/float(len(points)))))
    return new_center

#  更新聚类中心
def update_centers(data_set ,assignments,k):
    new_means = defaultdict(list)
    centers = []
    for assignment ,point in zip(assignments , data_set):
        new_means[assignment].append(point)
    for i in range(k):
        points=new_means[i]
        centers.append(point_avg(points))
    return centers

# 迭代 聚类
def final_result(data,assignments):
    old_assignments=None
    while assignments !=old_assignments:
        new_centers=update_centers(data,assignments,5)
        old_assignments=assignments
        assignments=assign_points(data,new_centers)
    
    return assignments

In [10]:
data1 = creat_data_1(table)

In [11]:
mm = mx_mn(data1)

In [12]:
data1 = redata(data1,mm)

In [13]:
#自定义聚类中心
k_points = [[1,1,0,0,0,0,0,0,0,0],[1,1,0,0,0,0,0,0.2,0.2,0.2],[1,1,1,1,0,0,0,0.4,0.4,0.4],[1,1,1,1,1,1,0,0.6,0.6,0.6],[1,1,1,1,1,1,1,0.8,0.8,0.8]]

In [14]:
assignments=assign_points(data1,k_points)

In [15]:
assignments = final_result(data1,assignments)

In [16]:
def count(ls):
    one = 0
    two = 0
    three = 0
    four = 0
    five = 0
    for i in ls:
        if i == 0:
            one+=1
        elif i==1:
            two+=1 
        elif i==2:
            three+=1 
        elif i==3:
            four+=1 
        else:
            five+=1 
    return one,two,three,four,five

In [17]:
one,two,three,four,five = count(assignments)

In [18]:
one,two,three,four,five

(7075, 100731, 5287, 890, 200)

In [19]:
key = table.col_values(0)[2:]

In [20]:
# 打包结果
label_key = table.col_values(0)[2:]
level_result=list(zip(key,assignments))

In [43]:
str(level_result[0][0])[:-2]

'199801010001'

In [47]:
level_results = []
for i in level_result:
    ll = []
    ll.append(str(i[0])[:-2])
    ll.append(i[1])
    level_results.append(ll)
    

In [48]:
from openpyxl import Workbook
wb = Workbook()
#创建一个sheet
ws = wb.create_sheet("data")
 

for i in level_results:
    ws.append(i)
    
    
wb.save('myExcelFile.xlsx')

In [20]:
#挑出5级破坏案件并评定破坏分数
def level_5_sc(level_result,data):
    lev_5 = {}
    for j,i in enumerate(level_result):
        if i[1] == 4:
            se_5 = [0,0]
            s= data[j][7]*3 + data[j][8]*2 + data[j][9]*2
            lev_5[i[0]] = s
    return lev_5

In [21]:
data_0 = creat_data_1(table)

In [22]:
_sc_5 = level_5_sc(level_result,data_0)

In [23]:
sc_sort = sorted(_sc_5.items(),key = lambda item:item[1],reverse = True)

In [24]:
sc_sort

[(200109110004.0, 35),
 (200109110005.0, 35),
 (200109110006.0, 30),
 (200409010002.0, 27),
 (201710140002.0, 27),
 (199808070002.0, 26),
 (200607120001.0, 26),
 (200708150005.0, 26),
 (200708160008.0, 26),
 (200908190001.0, 26),
 (200403210001.0, 25),
 (201607020002.0, 25),
 (201711240001.0, 25),
 (200108110012.0, 24),
 (200210120004.0, 24),
 (200210120005.0, 24),
 (200509140001.0, 24),
 (200703310010.0, 24),
 (200802010006.0, 24),
 (201404150089.0, 24),
 (200403110003.0, 23),
 (200403110007.0, 23),
 (200910250001.0, 23),
 (200910250005.0, 23),
 (199906190004.0, 22),
 (200308290001.0, 22),
 (200403020001.0, 22),
 (200502280002.0, 22),
 (200610160001.0, 22),
 (200611230001.0, 22),
 (200702030020.0, 22),
 (200703300001.0, 22),
 (200703300004.0, 22),
 (200704180001.0, 22),
 (200707070001.0, 22),
 (200710180001.0, 22),
 (200802170002.0, 22),
 (200910280029.0, 22),
 (201005280005.0, 22),
 (201007090005.0, 22),
 (201110040009.0, 22),
 (201411280012.0, 22),
 (201412160002.0, 22),
 (201504020

## 第二题

In [21]:
# 属性标签
attribute = [0,5,7,9,19,20,21,22,28,34,40,81,109]

In [22]:
# 抽取数据
def creat_data_2(table):
    data = []
    nrows = table.nrows
    for i in range(2,nrows):
        #获取一行数据
        rowvalue = table.row_values(i)
        if rowvalue[1] in [2015.0,2016.0] and rowvalue[71] == 0.0:
            e = []
            for j in attribute:
                e.append(rowvalue[j])
            data.append(e)
    return data

In [23]:
# 相似度 评分
def similar(data):
    similar_name = []
    for t,i in enumerate(data):
        szd = {23:[],22:[],21:[],20:[],19:[],18:[]}
        for n in data:
            if i[0] != n[0]:
                count = 0
                for j in range(1,len(attribute)):
                    if j == 1or j == 12:
                        if  i[j] == n[j] == 1:
                            count +=3
                        elif i[j] == n[j]:
                            count +=1
                    elif j == 8 or j == 10 or j == 11:
                        if i[j] == n[j]:
                            count +=2
                    elif j == 9:
                        count += 5
                    else:
                        if  i[j] == n[j]:
                            count += 1
                if count in szd.keys():
                    szd[count].append(n[0])
        if t%1000 ==0:
            print('.................'+str(t))
        similar_name.append(szd)
    return similar_name

In [47]:
# 相似案件挑选
def select(similar_name):
    se = []
    for w,i in enumerate(similar_name):

        if len(i[23])>0:
            se.append(i[23])
        if len(i[22])>0:
            se.append(i[22])
        if len(i[21])>0:
            se.append(i[21])
        if len(i[20])>0:
            se.append(i[20])
#         if len(i[19])>0:
#             se.append(i[19])
        else:
            se.append('')
    return se

In [25]:
data2 =  creat_data_2(table)

In [26]:
similar_name = similar(data2)

.................0
.................1000
.................2000
.................3000
.................4000
.................5000
.................6000
.................7000
.................8000
.................9000
.................10000
.................11000
.................12000
.................13000
.................14000
.................15000
.................16000
.................17000
.................18000
.................19000
.................20000
.................21000
.................22000


In [48]:
eve_sel = select(similar_name)

In [30]:
max_len = 0
for i in eve_sel:
    if len(i)>max_len:
        max_len = len(i)

In [33]:
#打包结果
def similar_data(data,eve_sel):
    s = {}
    for j,i in enumerate(data):
        s[data[j][0]] = eve_sel[j]
    return s

In [49]:
#打包结果
def similar_data(data,eve_sel):
    s = []
    for j,i in enumerate(data):
        ss = []
        ss.append(str(data[j][0])[:-2])
        for k in eve_sel[j]:
            ss.append(str(k)[:-2])
        s.append(ss)
    return s

In [50]:
sim_datas = similar_data(data2,eve_sel)

In [51]:
from openpyxl import Workbook
wb = Workbook()
#创建一个sheet
ws = wb.create_sheet("data")
 

for i in sim_datas:
    ws.append(i)
    
    
wb.save('File.xlsx')

In [240]:
# 15,16年事件 5,4 级破坏评分
def number(table,level_result,data):
    year = table.col_values(1)[2:]
    level_5 = {}
    level_4 = {}
    for i in range(len(year)):
        rowvalue = table.row_values(2+i)
        if year[i] in [2015.0,2016.0] and  rowvalue[71] == 0.0:
            if level_result[i][1] == 4:
                se_5 = [0,0]
                s= data[i][7]*3 + data[i][8]*2 + data[i][9]*2
                level_5[level_result[i][0]] = s
            elif level_result[i][1] == 3:
                se_4 = [0,0]
                s= data[i][7]*3 + data[i][8]*2 + data[i][9]*2
                level_4[level_result[i][0]] = s
    return level_5,level_4           

In [241]:
level_5,level_4 = number(table,level_result,data_0)

In [242]:
# 排序
level_5_sort = sorted(level_5.items(),key = lambda item:item[1],reverse = True)
level_4_sort = sorted(level_4.items(),key = lambda item:item[1],reverse = True)

In [243]:
level_5_sort

[(201510100002.0, 22),
 (201607140001.0, 21),
 (201612090017.0, 19),
 (201501300038.0, 18),
 (201507230037.0, 17),
 (201507230038.0, 17),
 (201601280038.0, 17),
 (201602090023.0, 17),
 (201608200001.0, 17),
 (201611230016.0, 17)]

In [244]:
def le_name(level_45_sort):
    l45_name = []
    for i in level_45_sort:
            l45_name.append(i[0])
    return l45_name

In [245]:
l5_name = le_name(level_5_sort)
l4_name = le_name(level_4_sort)

In [246]:
#  查看危害程度靠前的事件  是否有相似事件  如果有在危害列表中删除其相似的事件
def find_5(l5_name,l4_name,sim_datas):
    final = []
    for i in l5_name:
        final.append(i)
        for j in sim_datas[i]:
            if j in l5_name:
                l5_name.remove(j)
        if len(final) == 5:
            break
    if len(final) < 5:
        for i in l4_name:
            final.append(i)
            for j in sim_datas[i]:
                if j in l4_name:
                    l4_name.remove(j)
            if len(final) == 5:
                break
    return final      

In [247]:
final5_p = find_5(l5_name,l4_name,sim_datas)

In [248]:
final5_p

[201510100002.0,
 201607140001.0,
 201612090017.0,
 201501300038.0,
 201507230037.0]

In [230]:
final5_p

[201510100002.0,
 201607140001.0,
 201612090017.0,
 201501300038.0,
 201507230037.0]

In [228]:
aj_17 = [201701090031.0,
201702210037.0,
201703120023.0,
201705050009.0,
201705050010.0,
201707010028.0,
201707020006.0,
201708110018.0,
201711010006.0,
201712010003.0]

In [201]:
# 抽取数据
def creat_data_17(table,aj_17):
    data_cl = []
    data_js = []
    nrows = table.nrows
    for i in range(2,nrows):
        #获取一行数据
        rowvalue = table.row_values(i)
        if rowvalue[0] in aj_17:
            e = []
            for j in attribute:
                e.append(rowvalue[j])
            data_cl.append(e)
        elif rowvalue[0] in final5_p:
            e = []
            for j in attribute:
                e.append(rowvalue[j])
            data_js.append(e)
    return data_cl,data_js

In [202]:
data_17,data_js_5p = creat_data_17(table,aj_17)

In [256]:
data_17

[[201701090031.0,
  0.0,
  95.0,
  10.0,
  1.0,
  1.0,
  1.0,
  0.0,
  3.0,
  14.0,
  95.0,
  6.0,
  0.0],
 [201702210037.0, 0.0, 4.0, 6.0, 1.0, 1.0, 1.0, 0.0, 9.0, 3.0, 4.0, 13.0, 0.0],
 [201703120023.0,
  0.0,
  1004.0,
  11.0,
  1.0,
  1.0,
  1.0,
  0.0,
  7.0,
  4.0,
  1004.0,
  8.0,
  0.0],
 [201705050009.0,
  0.0,
  1004.0,
  11.0,
  1.0,
  1.0,
  1.0,
  0.0,
  2.0,
  19.0,
  1004.0,
  5.0,
  0.0],
 [201705050010.0,
  0.0,
  1004.0,
  11.0,
  1.0,
  1.0,
  1.0,
  0.0,
  2.0,
  19.0,
  1004.0,
  5.0,
  0.0],
 [201707010028.0,
  0.0,
  41.0,
  11.0,
  1.0,
  1.0,
  1.0,
  0.0,
  9.0,
  14.0,
  41.0,
  13.0,
  0.0],
 [201707020006.0,
  0.0,
  200.0,
  10.0,
  1.0,
  1.0,
  1.0,
  0.0,
  3.0,
  4.0,
  200.0,
  6.0,
  0.0],
 [201708110018.0, 0.0, 4.0, 6.0, 1.0, 1.0, 1.0, 0.0, 3.0, 14.0, 4.0, 6.0, 0.0],
 [201711010006.0, 0.0, 4.0, 6.0, 1.0, 1.0, 1.0, 0.0, 3.0, 19.0, 4.0, 6.0, 0.0],
 [201712010003.0,
  0.0,
  95.0,
  10.0,
  1.0,
  1.0,
  1.0,
  0.0,
  3.0,
  14.0,
  95.0,
  6.0,
  0.0]

In [210]:
data_js_5p

[[201501300038.0,
  0.0,
  37.0,
  11.0,
  1.0,
  1.0,
  0.0,
  1.0,
  3.0,
  4.0,
  42.0,
  6.0,
  0.0],
 [201507230037.0,
  0.0,
  95.0,
  10.0,
  1.0,
  1.0,
  1.0,
  0.0,
  3.0,
  4.0,
  95.0,
  6.0,
  0.0],
 [201510100002.0,
  0.0,
  209.0,
  10.0,
  1.0,
  1.0,
  1.0,
  0.0,
  3.0,
  14.0,
  209.0,
  6.0,
  0.0],
 [201607140001.0,
  0.0,
  69.0,
  8.0,
  1.0,
  1.0,
  1.0,
  0.0,
  2.0,
  14.0,
  69.0,
  5.0,
  0.0],
 [201612090017.0,
  0.0,
  147.0,
  11.0,
  1.0,
  1.0,
  1.0,
  0.0,
  3.0,
  14.0,
  147.0,
  6.0,
  0.0]]

In [257]:
# 相似度 评分
def similar_17(data_17,data_js_5p):
    similar_name = []
    for t,i in enumerate(data_17):
        szd = {23:[],22:[],21:[],20:[],19:[],18:[],17:[],16:[]}
        for n in data_js_5p:
            if i[0] != n[0]:
                count = 0
                for j in range(1,len(attribute)):
                    if j == 1or j == 12:
                        if  i[j] == n[j] == 1:
                            count +=3
                        elif i[j] == n[j]:
                            count +=1
                    elif j == 8 or j == 10 or j == 11:
                        if i[j] == n[j]:
                            count +=2
                    elif j == 9:
                        count += 5
                    else:
                        if  i[j] == n[j]:
                            count += 1
                if count in szd.keys():
                    szd[count].append(n[0])
        similar_name.append(szd)
    return similar_name

In [258]:
similar_17_name = similar_17(data_17,data_js_5p)

In [259]:
similar_17_name

[{16: [201510100002.0],
  17: [],
  18: [],
  19: [201507230037.0],
  20: [],
  21: [],
  22: [],
  23: []},
 {16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []},
 {16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []},
 {16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []},
 {16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []},
 {16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []},
 {16: [201507230037.0, 201510100002.0],
  17: [],
  18: [],
  19: [],
  20: [],
  21: [],
  22: [],
  23: []},
 {16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []},
 {16: [], 17: [], 18: [], 19: [], 20: [], 21: [], 22: [], 23: []},
 {16: [201510100002.0],
  17: [],
  18: [],
  19: [201507230037.0],
  20: [],
  21: [],
  22: [],
  23: []}]